In [133]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA
from sklearn.cluster import SpectralClustering

In [74]:
def totalAnalysis(fileName):
    data = pd.read_csv(fileName)
    realData = data.values
    foo = [x[0] for x in realData]
    print('Number of IDs : ', len(set(foo)))
    bar = [x[1] for x in realData]
    bar = np.array(list(set(bar)))
    print('Number of Months : ', bar.size)
    print(bar)
    
def monthAnalysis(fileName):
    data = pd.read_csv(fileName)
    realData = data.values
    InterMonthData = {}
    IntraMonthData = {}
    MonthZeroData = {}
    for x in realData:
        if x[6] !=0 :
            if x[1] not in InterMonthData:
                InterMonthData[x[1]] = [x[0]]
            else :
                InterMonthData[x[1]].append(x[0])
        else :
            if x[1] not in IntraMonthData:
                IntraMonthData[x[1]] = [x[0]]
            else :
                IntraMonthData[x[1]].append(x[0])
        if x[2] ==0:
            if x[1] not in MonthZeroData : 
                MonthZeroData[x[1]] = [x[0]]
            else :
                MonthZeroData[x[1]].append(x[0])
    print('Inter Monthwise entries: ')
    for key in InterMonthData:
        print(key, len(InterMonthData[key]))
    print('Intra Monthwise entries: ')
    for key in IntraMonthData:
        print(key, len(IntraMonthData[key]))
    print('Monthwise zero entries: ')
    for key in MonthZeroData:
        print(key, len(MonthZeroData[key]))

In [75]:
totalAnalysis('data_class.csv')
monthAnalysis('data_class.csv')

Number of IDs :  11201
Number of Months :  12
[102017.  62018.  82017.  22018. 122017.  42018.  92017.  32018.  52018.
  12018.  72017. 112017.]
Inter Monthwise entries: 
32018.0 3215
52018.0 2921
12018.0 3055
92017.0 2902
102017.0 2885
62018.0 2863
82017.0 2848
22018.0 3129
122017.0 3048
42018.0 2943
72017.0 2291
112017.0 3002
Intra Monthwise entries: 
42018.0 4643
92017.0 5926
82017.0 6184
72017.0 6786
102017.0 5685
32018.0 4935
122017.0 5080
62018.0 4337
12018.0 4966
52018.0 4553
22018.0 4901
112017.0 5212
Monthwise zero entries: 
92017.0 1031
82017.0 1415
72017.0 2297
102017.0 788
32018.0 270
122017.0 247
62018.0 253
52018.0 292
112017.0 368
12018.0 248
22018.0 335
42018.0 307


In [135]:
def getMonthlyData(fileName):
    data = pd.read_csv(fileName)
    realData = data.values
    WholeData = {}
    NumData = {}
    MonthMap = { 72017: 1, 82017:2, 92017:3, 102017:4, 112017:5, 122017:6, 12018:7, 22018:8, 32018:9, 42018:10, 52018:11, 62018:12}
    for x in realData:
        if x[2] !=0:
            y = MonthMap[x[1]]
            if y not in WholeData : 
                WholeData[y] = [x]
                NumData[y] = [x[0]]
            else :
                WholeData[y].append(x)
                NumData[y].append(x[0])
    return (NumData, WholeData)

def getLowDimensionData(Data):
    pca = PCA(n_components = 2)
    return pca.fit_transform(Data)

def distanceMetric(entry1, entry2):
    ans = np.array([(entry1[cnt]- entry2[cnt])**2 for cnt in range(len(entry1))])
    return np.sum(ans)

def getAdjacencyMatrix(Data):
    AdjacencyMatrix = []
    for x in Data:
        temp = []
        for y in Data:
            ans = distanceMetric(x, y)
            temp.append(ans)
        AdjacencyMatrix.append(temp)
    return AdjacencyMatrix

In [136]:
(NumData, WholeData) = getMonthlyData('data_class.csv')

In [139]:
# Unprocessed Raw Data
for month in range(1, 13):
    UnMonthData = np.array(WholeData[month], dtype = np.int32)
    # Get the first month Data and MonthId Data
    MonthData = [x[2:] for x in UnMonthData]
    MonthIdData = np.array(NumData[month], dtype = np.int32)
    AdjacencyMatrix = getAdjacencyMatrix(MonthData)
    LowMonthData = getLowDimensionData(MonthData)
    clustering = SpectralClustering(n_clusters=2,
        assign_labels="discretize",
        random_state=0).fit(AdjacencyMatrix)
    print(clustering.labels_)
    print("Month ", month, " completed")
#     plot of 2D data 
#     X  = [x[0] for x in LowMonthData]
#     Y = [x[1] for x in LowMonthData]
#     plt.plot(X, Y, 'r.')
#     plt.show()

/Users/re1nth/anaconda3/lib/python3.7/site-packages/sklearn/cluster/spectral.py:462: UserWarning: The spectral clustering API has changed. ``fit``now constructs an affinity matrix from data. To use a custom affinity matrix, set ``affinity=precomputed``.
  warnings.warn("The spectral clustering API has changed. ``fit``"
/Users/re1nth/anaconda3/lib/python3.7/site-packages/sklearn/manifold/spectral_embedding_.py:237: UserWarning: Graph is not fully connected, spectral embedding may not work as expected.
  warnings.warn("Graph is not fully connected, spectral embedding"


[1 0 0 ... 1 0 0]
Month  1  completed


/Users/re1nth/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:23: RuntimeWarning: overflow encountered in int_scalars


KeyboardInterrupt: 

/Users/re1nth/anaconda3/lib/python3.7/site-packages/sklearn/manifold/spectral_embedding_.py:237: UserWarning: Graph is not fully connected, spectral embedding may not work as expected.
  warnings.warn("Graph is not fully connected, spectral embedding"
/Users/re1nth/anaconda3/lib/python3.7/site-packages/sklearn/cluster/spectral.py:99: RuntimeWarning: divide by zero encountered in true_divide
  vectors = vectors / np.sqrt((vectors ** 2).sum(axis=1))[:, np.newaxis]
/Users/re1nth/anaconda3/lib/python3.7/site-packages/sklearn/cluster/spectral.py:99: RuntimeWarning: invalid value encountered in true_divide
  vectors = vectors / np.sqrt((vectors ** 2).sum(axis=1))[:, np.newaxis]


In [ ]:
print(clustering.labels_)